In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-product-reviews/Reviews.csv


In [2]:
!pip install datasets transformers evaluate huggingface_hub

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
import numpy as np
import evaluate

In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
pipe("I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': '5 stars', 'score': 0.7185351848602295}]

In [6]:
dataset = load_dataset('csv', data_files='/kaggle/input/amazon-product-reviews/Reviews.csv')
print("Columns before removal:", dataset['train'].column_names)

Columns before removal: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text']


In [7]:
columns_to_remove = [
    'Id', 'ProductId', 'UserId', 'ProfileName',
    'HelpfulnessNumerator', 'HelpfulnessDenominator',
    'Time', 'Summary'
]
dataset = dataset.remove_columns(columns_to_remove)

In [8]:
train_test = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test['train']
test_dataset = train_test['test']

In [9]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [10]:
def preprocess_function(examples):
    result = tokenizer(
        examples["Text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    result['labels'] = [score - 1 for score in examples['Score']]
    return result

In [11]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/454763 [00:00<?, ? examples/s]

Map:   0%|          | 0/113691 [00:00<?, ? examples/s]

In [12]:
tokenized_train_dataset[0]

{'Score': 3,
 'Text': 'It does what it is supposed to, but I was expecting to get a lot more than we did.  I think I will try and make my own in the future.',
 'input_ids': [101,
  10197,
  14893,
  11523,
  10197,
  10127,
  54838,
  10114,
  117,
  10502,
  151,
  10140,
  11460,
  84789,
  10285,
  10114,
  13168,
  143,
  15632,
  10772,
  10948,
  11312,
  12266,
  119,
  151,
  21506,
  151,
  11229,
  26333,
  10110,
  12696,
  11153,
  12530,
  10104,
  10103,
  14299,
  119,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,


In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
accuracy = evaluate.load("accuracy")

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Convert logits to predicted class indices and adjust back to 1-5 scale
    predictions = np.argmax(predictions, axis=1) + 1
    labels = labels + 1  # Adjust labels back to original scale for accuracy
    return accuracy.compute(predictions=predictions, references=labels)


In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment",
    num_labels=5
)

In [16]:
!pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.7 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [17]:
from huggingface_hub import login

token = ""
login(token)



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
training_args = TrainingArguments(
    output_dir="SentimentAnalysisAtDEPI",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
    push_to_hub=False  
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.558300,0.541303,0.795789
2,0.493900,0.500668,0.820126
3,0.456400,0.492281,0.831438


TrainOutput(global_step=42636, training_loss=0.49523145298157506, metrics={'train_runtime': 24380.5339, 'train_samples_per_second': 55.958, 'train_steps_per_second': 1.749, 'total_flos': 8.974229686601702e+16, 'train_loss': 0.49523145298157506, 'epoch': 3.0})

In [21]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdelrahmanelsheikh39/SentimentAnalysisAtDEPI/commit/ca1a1a85b237b6ebf31f70ecc005c86947151c30', commit_message='End of training', commit_description='', oid='ca1a1a85b237b6ebf31f70ecc005c86947151c30', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abdelrahmanelsheikh39/SentimentAnalysisAtDEPI', endpoint='https://huggingface.co', repo_type='model', repo_id='abdelrahmanelsheikh39/SentimentAnalysisAtDEPI'), pr_revision=None, pr_num=None)

In [22]:
text ="I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most."

In [23]:
from transformers import pipeline
classifier = pipeline("text-classification", model="abdelrahmanelsheikh39/SentimentAnalysisAtDEPI")
classifier(text)

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': '5 stars', 'score': 0.9597824215888977}]